# <span style="color:#A27619">Проекст: Украинско-Русский переводчик на TF-IDF</span>

### План проекта

1. [Чтение данных](#section1)
2. [Готовая модель линейной регрессии](#section2)
3. [SVD - Singular Value Decomposition](#section3)
4. [Переводчик](#section4)

### Импорт зависимостей

In [9]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

from sklearn.linear_model import LinearRegression

<a id='section1'></a>

## <span style="color:#1ABC9C">1. Загрузка данных</span>

In [2]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [3]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240028262138367),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.893003523349762),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401415944099426)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('липень', 0.9096439480781555),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810407519340515),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804074287415),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256802558898926),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.2200372815132141),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

По готовому словарю слов сформируем словарь эмбедингов:

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, encoding = 'utf-8', mode= "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [13]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [14]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

<a id='section2'></a>

## <span style="color:#1ABC9C">2. Готовая модель линейной регрессии</span>

In [15]:
mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression()

Посмотрим, какой получилась модель. Как она переведет украинское слово "срепень" - "август" по-русски.

In [16]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541591763496399),
 ('июнь', 0.8411964178085327),
 ('март', 0.8397400379180908),
 ('сентябрь', 0.8359215259552002),
 ('февраль', 0.8328748941421509),
 ('октябрь', 0.8311805725097656),
 ('ноябрь', 0.827814519405365),
 ('июль', 0.8236350417137146),
 ('август', 0.8120612502098083),
 ('декабрь', 0.8038000464439392)]

Вроде месяца. Но нужный только на 9-м месте.

Объявим метрику качества. Она будет считать, в какой topn попало действительное значение и сколько было таких попаданий среди всех введенных пар:

In [18]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        if ru in [item[0] for item in ru_emb.most_similar(mapped_vectors[i])[:topn]]:
            num_matches+=1
    precision_val = num_matches / len(pairs)
    return precision_val


In [19]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [20]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [22]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

print('Для топ 1:', precision_top1)
print('Для топ 5:', precision_top5)

Для топ 1: 0.6356589147286822
Для топ 5: 0.8113695090439277


<span style="color:#CC6600">**Заключение**</span>
- на данный момент модель правильно переводит только 63.5% от всех слов.

<a id='section3'></a>

## <span style="color:#1ABC9C">3. SVD - singular value decomposition</span>

Любую матрицу можно разложить на 3 составляющие.

In [23]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, S, Vt = np.linalg.svd(X_train.T @ Y_train)
    return U @ Vt

In [24]:
W = learn_transform(X_train, Y_train)

In [25]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237908482551575),
 ('сентябрь', 0.8049712181091309),
 ('март', 0.802565336227417),
 ('июнь', 0.8021840453147888),
 ('октябрь', 0.8001735210418701),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914118766784668),
 ('июль', 0.7908107042312622),
 ('август', 0.7891014814376831),
 ('декабрь', 0.7686371803283691)]

In [26]:
precision_top1 = precision(uk_ru_test, np.matmul(X_test, W), 1)
precision_top5 = precision(uk_ru_test, np.matmul(X_test, W), 5)

print('Для топ 1:', precision_top1)
print('Для топ 5:', precision_top5)

Для топ 1: 0.6537467700258398
Для топ 5: 0.8242894056847545


<span style="color:#CC6600">**Заключение**</span>
- теперь модель правильно переводит 65.3% всех слов (стало немного лучше)

<a id='section4'></a>

## <span style="color:#1ABC9C">4. Переводчик</span>

In [27]:
with open("fairy_tale.txt", encoding = 'utf-8', mode= "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [28]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    embs = []
    for word in sentence.split():
        if word in uk_emb:
            embs.append(uk_emb[word])
        #else:
        #    embs.append(-10000)

    sent = []
    for emb in embs:
        ru_word = ru_emb.most_similar([np.matmul(emb, W)])[0][0]
        sent.append(ru_word)
    return ' '.join(sent)

In [29]:
translate("кіт зловив мишу")

'кот поймал мышку'

In [30]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк –

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала и и а оно приходят из лисичка замерзла и и побежала во село огня чтобы прибегает к одной бабы и и

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые со мне мной тебе

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — — — – садись пока мной пирожки со

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала сміттячка, стулила та й біжить.
dst: а бабка маковые пирожки из бабка выбирает пирожки и по столе чтобы а лисичка и за и со со а туда и и

src: от біжить, а хлопці товар ж